In [1]:
import os
import sys
import glob
import pandas as pd
import shutil

In [12]:
# === Configuration ===
ROOT_PATH = '/Users/alexandergiberson/Documents/Masters_Work/WRM_TWG_CREDIT_RUNS/Running_Credits/Cap - 850'
EXCEL_FILENAME = "cap_Canada_Take_deficit_change.xlsx"
SHEET_NAME = "Capped_Credit"
HBDF_FILENAME = "hbdf.txt"
NEW_HBDF_FILENAME = "New_US_MILK_HBDF.txt"
OUTSIM_FILENAME = "outsim.csv"
OUTSIM_BACKUP_PREFIX = "outsim_milk_run_"
BACKUP_EXT = ".csv"
ROW1 = 1081
ROW2 = 1116


In [2]:
# === Function Definitions ===

def get_next_run_number(base_dir, prefix="hbdf_milk_run_", ext=".txt"):
    pattern = os.path.join(base_dir, f"{prefix}*{ext}")
    existing = glob.glob(pattern)
    nums = []
    for path in existing:
        name = os.path.basename(path)
        try:
            num = int(name[len(prefix):-len(ext)])
            nums.append(num)
        except ValueError:
            continue
    n = 1
    while n in nums:
        n += 1
    return n

def replace_section_with_backup(original_path, new_data_path, backup_prefix="hbdf_milk_run_"):
    base_dir = os.path.dirname(os.path.abspath(original_path))
    run_num = get_next_run_number(base_dir, backup_prefix)
    backup_name = f"{backup_prefix}{run_num}.txt"
    backup_path = os.path.join(base_dir, backup_name)

    with open(original_path, 'r') as f:
        lines = f.readlines()
    with open(new_data_path, 'r') as f:
        new_lines = f.readlines()

    header_idx = next(
        (i for i, line in enumerate(lines)
         if line.startswith(" NATCH984") and "1981" in line and "CMS" in line),
        None
    )
    if header_idx is None:
        sys.exit("ERROR: Header 'NATCH984  1981  35 CMS' not found.")

    year_idx = next(
        (j for j in range(header_idx + 1, len(lines))
         if lines[j].lstrip().startswith("Year")),
        None
    )
    if year_idx is None:
        sys.exit("ERROR: 'Year ...' line not found after header.")

    data_start = year_idx + 1
    data_end = data_start
    while data_end < len(lines):
        tok = lines[data_end].strip().split()
        if not tok or not tok[0].isdigit() or len(tok[0]) != 4:
            break
        data_end += 1

    updated = lines[:data_start] + new_lines + lines[data_end:]

    os.rename(original_path, backup_path)
    print(f"✔️  Backed up original to '{backup_name}'")

    with open(original_path, 'w') as f:
        f.writelines(updated)
    print("✔️  Wrote updated contents to 'hbdf.txt'")

def generate_formatted_hbdf(input_path, sheet_name, output_path, row1, row2):
    df = pd.read_excel(input_path, sheet_name=sheet_name)
    flow_data = df.iloc[row1:row2, 2:55]

    formatted_lines = []
    for _, row in flow_data.iterrows():
        year = f"{int(row.iloc[0]):4d}"
        flow_parts = []
        for val in row.iloc[1:]:
            formatted_val = f"{val:9.3f}"  # Always 9-character wide, right-aligned
            flow_parts.append(formatted_val)
        line = f"{year} " + ''.join(flow_parts)
        formatted_lines.append(line)

    with open(output_path, "w") as f:
        for line in formatted_lines:
            f.write(line + "\n")

def backup_outsim():
    base_dir = os.path.abspath(ROOT_PATH)
    run_num = get_next_run_number(base_dir, OUTSIM_BACKUP_PREFIX, BACKUP_EXT)
    src_path = os.path.join(base_dir, OUTSIM_FILENAME)
    if not os.path.exists(src_path):
        raise FileNotFoundError(f"outsim file not found: {src_path}")

    backup_name = f"{OUTSIM_BACKUP_PREFIX}{run_num}{BACKUP_EXT}"
    backup_path = os.path.join(base_dir, backup_name)
    shutil.copyfile(src_path, backup_path)
    print(f"✔️  Copied '{OUTSIM_FILENAME}' to '{backup_name}'")
    

In [8]:
# === Configuration ===
ROOT_PATH = '/Users/alexandergiberson/Documents/Masters_Work/WRM_TWG_CREDIT_RUNS/Capped_Credit_Run_600cfs'
EXCEL_FILENAME = "cap_Canada_Take.xlsx"
SHEET_NAME = "Capped_Credit"
HBDF_FILENAME = "hbdf.txt"
NEW_HBDF_FILENAME = "New_US_MILK_HBDF.txt"
OUTSIM_FILENAME = "outsim.csv"
OUTSIM_BACKUP_PREFIX = "outsim_milk_run_"
BACKUP_EXT = ".csv"
ROW1 = 1081
ROW2 = 1116


In [13]:
# === Main Execution ===
if __name__ == "__main__":
    excel_path = os.path.join(ROOT_PATH, EXCEL_FILENAME)
    new_hbdf_path = os.path.join(ROOT_PATH, NEW_HBDF_FILENAME)
    old_hbdf_path = os.path.join(ROOT_PATH, HBDF_FILENAME)

    generate_formatted_hbdf(excel_path, SHEET_NAME, new_hbdf_path, ROW1, ROW2)

    replace_section_with_backup(
        original_path=old_hbdf_path,
        new_data_path=new_hbdf_path
    )
    backup_outsim()
    
    

✔️  Backed up original to 'hbdf_milk_run_5.txt'
✔️  Wrote updated contents to 'hbdf.txt'
✔️  Copied 'outsim.csv' to 'outsim_milk_run_5.csv'
